In [31]:
import cv2

# test image
image = cv2.imread('img_simi/1.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
histogram = cv2.calcHist([gray_image], [0],None, [256], [0, 256])

# data1 image
image = cv2.imread('img_simi/2.jpg')
gray_image1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
histogram1 = cv2.calcHist([gray_image1], [0],None, [256], [0, 256])

# data2 image
image = cv2.imread('img_simi/3.jpg')
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
histogram2 = cv2.calcHist([gray_image2], [0],None, [256], [0, 256])

c1, c2 = 0, 0
# Euclidean Distace between data1 and test
i = 0
while i<len(histogram) and i<len(histogram1):
	c1+=(histogram[i]-histogram1[i])**2
	i+= 1
c1 = c1**(1 / 2)

# Euclidean Distace between data2 and test
i = 0
while i<len(histogram) and i<len(histogram2):
	c2+=(histogram[i]-histogram2[i])**2
	i+= 1
c2 = c2**(1 / 2)
if(c1<c2):
	print("2.jpg is more similar to 1.jpg as compare to 3.jpg")
else:
	print("3.jpg is more similar to 1.jpg as compare to 2.jpg")

2.jpg is more similar to 1.jpg as compare to 3.jpg


In [37]:
from PIL import Image
import imagehash
hash0 = imagehash.average_hash(Image.open('img_simi/1.jpg')) 
hash1 = imagehash.average_hash(Image.open('img_simi/6.jpg')) 
cutoff = 1     # maximum bits that could be different between the hashes. 

if hash0 - hash1 <= cutoff:
  print('images are similar')
else:
  print('images are not similar')

images are not similar


In [38]:
# Based on: https://github.com/mostafaGwely/Structural-Similarity-Index-SSIM-

# 1. Import the necessary packages
from skimage.metrics import structural_similarity
import cv2

# 2. Construct the argument parse and parse the arguments

# 3. Load the two input images
imageA = cv2.imread("img_simi/1.jpg")
imageB = cv2.imread("img_simi/img4.jpg")

# 4. Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# 5. Compute the Structural Similarity Index (SSIM) between the two
#    images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# 6. You can print only the score if you want
print("SSIM: {}".format(score))

SSIM: 0.9548491607866954


In [30]:
from skimage.metrics import structural_similarity
import cv2

#Works well with images of different dimensions
def orb_sim(img1, img2):
  # SIFT is no longer available in cv2 so using ORB
  orb = cv2.ORB_create()

  # detect keypoints and descriptors
  kp_a, desc_a = orb.detectAndCompute(img1, None)
  kp_b, desc_b = orb.detectAndCompute(img2, None)

  # define the bruteforce matcher object
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
  #perform matches. 
  matches = bf.match(desc_a, desc_b)
  #Look for similar regions with distance < 50. Goes from 0 to 100 so pick a number between.
  similar_regions = [i for i in matches if i.distance < 50]  
  if len(matches) == 0:
    return 0
  return len(similar_regions) / len(matches)


#Needs images to be same dimensions
def structural_sim(img1, img2):

  sim, diff = structural_similarity(img1, img2, full=True)
  return sim

img00 = cv2.imread('img_simi/1.jpg', 0)
img01 = cv2.imread('img_simi/2.jpg', 0)

img1 = cv2.imread('img_simi/3.jpg', 0)  # 714 x 901 pixels
img2 = cv2.imread('img_simi/4.jpg', 0)  # 714 x 901 pixels
img3 = cv2.imread('img_simi/5.jpg', 0)  # 203 x 256 pixels
img4 = cv2.imread('img_simi/6.jpg', 0)  # 203 x 256 pixels

orb_similarity = orb_sim(img1, img3)  #1.0 means identical. Lower = not similar

print("Similarity using ORB is: ", orb_similarity)
#Resize for SSIM
from skimage.transform import resize
img5 = resize(img3, (img1.shape[0], img1.shape[1]), anti_aliasing=True, preserve_range=True)

ssim = structural_sim(img1, img5) #1.0 means identical. Lower = not similar
print("Similarity using SSIM is: ", ssim)


Similarity using ORB is:  0.08461538461538462
Similarity using SSIM is:  0.11373466099317998


/home/akshay_kanawade/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
